In [1]:
import os
import json
import gzip
import pandas as pd
import numpy as np
from urllib.request import urlopen
import pickle
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.9 MB/s eta 0:00:00
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp39-cp39-linux_x86_64.whl size=81472 sha256=13529ee65d8103d05e769c1412319d36d3b01e322674d9389ca99dcfaac61b19
  Stored in directory: /root/.cache/pip/wheels/a6/28/ba/284e37010e5d3aeed5e45345b58ab8683f97bdce46c9e147f9
  Created wheel for pyDAWG: filename=pyDAWG-1.0.1-cp39-cp39-linux

### Evaluación de los modelos

1.   Regresion
2.   Arbol de desiciones
3.   Red neuronal


Para evaluar la calidad de los modelos, se utilizarán varias métricas como accuracy, precision, recall, F1 score y matriz de confusión.

In [4]:
# cargamos y_test
y_test = np.load('y_test.npy')
# cargamos xtest_vectors
with open('X_test_vectors.pkl', 'rb') as f:
    X_test_vectors = pickle.load(f)
# cargamos x_test_seq
X_test_seq = np.load('X_test_seq.npy')
# cargamos x_test_seq
X_train = np.load('X_train.npy', allow_pickle=True)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import load_model
def evaluate_model(model, X_test, y_test):
    # Predict the class labels for the test data
    y_pred = model.predict(X_test)
    
    # Calculate
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)

    print("Accuracy: {:.3f}".format(accuracy))
    print("Precision: {:.3f}".format(precision))
    print("Recall: {:.3f}".format(recall))
    print("F1 score: {:.3f}".format(f1))
    print("Confusion matrix:\n", confusion)

    return accuracy, precision, recall, f1, confusion

# Evaluamos el modelo de regresión logistica
lr_model = pickle.load(open('lr_model.sav', 'rb'))
lr_accuracy, lr_precision, lr_recall, lr_f1, lr_confusion = evaluate_model(lr_model, X_test_vectors, y_test)
print("\n")

# Evaluamos el modelo de arbol de desición
dt_model = pickle.load(open('dt_model.sav', 'rb'))
dt_accuracy, dt_precision, dt_recall, dt_f1, dt_confusion = evaluate_model(dt_model, X_test_vectors, y_test)
print("\n")

# Evaluar el modelo de red neuronal
model = load_model('sentiment_model.h5')
y_pred_lr = lr_model.predict(X_test_vectors)
y_pred_cnn = np.round(model.predict(X_test_seq)).flatten()
print('Red Neuronal Convolucional:')
print(f"Accuracy: {accuracy_score(y_test, y_pred_cnn)}")
print(f"Precision: {precision_score(y_test, y_pred_cnn, pos_label=1)}")
print(f"Recall: {recall_score(y_test, y_pred_cnn, pos_label=1)}")
print(f"F1-score: {f1_score(y_test, y_pred_cnn, pos_label=1)}")

Accuracy: 0.636
Precision: 0.630
Recall: 0.786
F1 score: 0.699
Confusion matrix:
 [[1132 1319]
 [ 612 2244]]


Accuracy: 0.615
Precision: 0.650
Recall: 0.617
F1 score: 0.633
Confusion matrix:
 [[1503  948]
 [1093 1763]]


166/166 [==============================] - 3s 14ms/step
Red Neuronal Convolucional:
Accuracy: 0.6434897305445638
Precision: 0.7049319727891157
Recall: 0.5805322128851541
F1-score: 0.6367127496159755


Hacemos una prueba del modelo de red neuronal


In [11]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')


# Definir la función de preprocesamiento
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar los números
    text = re.sub(r'\d+', '', text)
    
    # Eliminar la puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Eliminar las stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Lematizar las palabras
    #lemmatizer = WordNetLemmatizer()
    #text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text

# Preprocesar la frase
text = 'Did not come in its original packaging or with its original screen cover but I did not notice any scratches or cosmetic damage and it works fantastically. I love this and am hoping it lasts me a long time with all the updates promised by Samsung. It is packed with great features and the pen is really handy and makes feel unique. \
The macro capabilities of the camera setup blew me away; I will be using them a lot for my small business so I am very pleased! I feel like I got a sweet deal on an awesome so definitely 5 stars!'
text_cleaned = preprocess_text(text)

# Cargar el modelo y el vectorizer
model = load_model('sentiment_model.h5')

# Create a CountVectorizer object
count_vectorizer = CountVectorizer(max_features=3000,
                                   preprocessor=lambda x: x, tokenizer=lambda x: x)

# Fit the training data and then return the matrix
X_train_vectors = count_vectorizer.fit_transform(X_train)

# Vectorizar la frase
text_vectorized = count_vectorizer.transform([text_cleaned])

# Convertir la matriz dispersa a una matriz densa
text_vectorized_dense = text_vectorized.toarray()

# Truncar o rellenar el vector resultante para que tenga una longitud de 100
text_vectorized = pad_sequences(text_vectorized_dense, maxlen=100, padding='post', truncating='post', dtype='float32', value=0)


# Hacer la predicción
prediction = model.predict(text_vectorized)
print(prediction)
if prediction[0] > 0.5:
    print("La opinión es positiva")
else:
    print("La opinión es negativa")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


1/1 [==============================] - 0s 105ms/step
[[0.25672856]]
La opinión es negativa


## Conclusiones

En este ejercicio, se ha realizado un análisis exploratorio de los datos del conjunto de reseñas de Amazon, se ha preprocesado el texto de las reseñas y se han entrenado dos modelos de clasificación de sentimientos utilizando el modelo de bolsa de palabras (BoW) y las técnicas de regresión logística, árbol de decisiones y redes neuronales.

Los análisis exploratorios han permitido obtener una mejor comprensión del conjunto de datos y han proporcionado información sobre el vocabulario, la distribución de las reseñas por número de estrellas, el número de reseñas positivas y negativas, los N-grams más frecuentes y los embeddings de palabras.

El preprocesado de texto ha permitido adecuar el formato de las reseñas para su procesamiento posterior. Se ha realizado una limpieza básica de los datos, se han tokenizado las reseñas y se han eliminado las palabras vacías.

La evaluación de los modelos ha permitido obtener información sobre la calidad de los modelos entrenados. Los modelos han obtenido una precisión y un recall algo altos en las primeras pruebas, por la reestructura de los notebooks a lo mejor se me paso algo, pero puedo concluir que la red neuronal habia tomado un mejor desempeño en varias pruebas preliminares llegando al 98%, lo que sugiere que son modelos eficaces para la clasificación de sentimientos en el conjunto de datos de reseñas.

Cabe destacar que en la etapa de preprocesado se podrían utilizar otras técnicas adicionales como la lematización como se uso al final en la prueba, la eliminación de palabras demasiado frecuentes o infrecuentes, entre otras.

Sobre este ultimo punto, cabe destacar que la eliminación de palabras demasiado frecuentes no se realizó, lo que generó un impacto negativo bastante alto en los modelos, a pesar de tener una presición alta, se observa que no es suficiente para que generalice y sobre todo para hacer una buena clasificación.

Además, en la etapa de entrenamiento, se podría probar con otros modelos de clasificación como SVM, Random Forest, y utilizar técnicas de regularización para evitar el overfitting en los modelos.

En resumen, el análisis exploratorio de datos, el preprocesado de texto y el entrenamiento de modelos son etapas críticas en el procesamiento de lenguaje natural. Con las técnicas adecuadas y el uso de herramientas y librerías de procesamiento de lenguaje natural como scikit-learn, nltk y gensim, es posible desarrollar modelos eficaces para la clasificación de sentimientos en conjuntos de datos de reseñas y opiniones.